<img src="http://oci02.img.iteso.mx/identidad_de_instancia_2018/ITESO/Logos%20ITESO/Logo-ITESO-Principal.jpg" width="800px" height="400px"/>

# <font color='midnightblue'> <center> Concurso Interuniversitario BMV-Peeptrade </center>

# <font color='midnightblue'> **Integrantes:**

<font color='midnightblue'> **Rubén Hernández Guevara | if717710**
    
<font color='midnightblue'> **Juan Pablo Rodríguez Alonso | if715180**
    
<font color='midnightblue'> **Andrés Ramírez Villanueva | if726061**

In [1]:
import matplotlib
import numpy as np
import pandas as pd
import scipy.stats as stats
import plotly.express as px
import scipy.optimize as opt
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import pandas_datareader.data as web
from plotly.subplots import make_subplots
from pandas_datareader import data as pdr
import yfinance as yfin
yfin.pdr_override()

In [51]:
def simulacion_pfolios(ret: 'Retornos',
                       rf: 'Tasa Libre de Riesgo',
                       tickers: 'Ticker de los activos'):
    
    global pesos, w_EMV, w_opt, E_EMV, s_EMV, summary, S, Sigma, Eind, cov, corr
        
    #Rendimientos y volatilidades esperadas
    summary = pd.DataFrame(data={'Media': ret.mean(),
                                 'Vol':ret.std()})
        
    cov_ret = ret.cov() # Matriz de varianza - covarianza
    
    corr = ret.corr() # Matriz de correlación
    
    S = np.diag(summary['Vol'].values)
    Sigma = S.dot(corr).dot(S)
    Eind = summary['Media'].values
    
    n = len(Eind) # Número de activos    
    w0 = np.ones((n,))/n # Dato inicial   
    bnds = ((0, .1),)*n # Cotas de las variables    
    cons={'type':'eq','fun':lambda w:w.sum()-1} # Restricciones
    
    # Portafolio EMV
    EMV = minimize(fun = menos_RS,
                   x0 = w0,
                   args = (Eind, Sigma, rf),
                   bounds = bnds,
                   constraints = cons,
                   tol = 1e-5)
    w_EMV = EMV.x
    w_EMV = np.round(w_EMV,4)
    
    # Pesos, rendimiento y riesgo del portafolio EMV
    E_EMV = Eind.T.dot(w_EMV)
    s_EMV = (w_EMV.T.dot(Sigma).dot(w_EMV))**0.5
    RS_EMV = (E_EMV - rf)/s_EMV
    
    summary['w'] = np.round(w_EMV,4)
    
    pfolio = pd.DataFrame({'E_EMV': [E_EMV],
                           'Volatilidad': [s_EMV],
                           'Sharpe Ratio': [RS_EMV]})
    
    return summary, pfolio

In [3]:
def simulacion_inversion(inversion: 'Dinero Invertido',
                         precios: 'Precios del Portafolio',
                         pesos: 'Pesos de Activos en Portafolio',
                         retornos: 'Retornos del Portafolio'):
    
    global inv_real, inv_teor, asignacion_activos, asignacion_rf, vfinal_activos, vfinal_rf, vfinal_pfolio,\
    rnd_activos, rnd_rf, rnd_pfolio, cumrets, Resultados
    
    asignacion_activos = np.sum(pesos[0:15])*inversion
    asignacion_rf = inversion - asignacion_activos
    
    inv_teor = [pesos[i]*inversion for i in range(15)] # Inversión Teórica de Acuerdo a las Ponderaciones. 
    
    vi_acciones = [precios.iloc[0].values[i] for i in range(15)] # Valor Inicial Real de las Acciones Al Momento de la Inversión.
    
    # Número de acciones a comprar
    n_acciones = [(inv_teor[i]/vi_acciones[i]) for i in range(15)]
    n_acciones = np.round(n_acciones) 
    
    inv_real = [n_acciones[i]*vi_acciones[i] for i in range(15)] # Inversión real, de acuerdo a n acciones y precio inicial
    
    restante_inv_rf = [inv_teor[i]-inv_real[i] for i in range(15)]
    restante_inv_rf = np.sum(restante_inv_rf)
    asignacion_rf += restante_inv_rf
    
    
    cumrets = pd.DataFrame(data = {'Date': retornos['Date'],
                               'VNQ': (retornos['VNQ'] + 1).cumprod()*(inv_real[0]),
                               'VBK': (retornos['VBK'] + 1).cumprod()*(inv_real[1]),
                               'SPY': (retornos['SPY'] + 1).cumprod()*(inv_real[2]),
                               'VTI': (retornos['VTI'] + 1).cumprod()*(inv_real[3]),
                               'VGT': (retornos['VGT'] + 1).cumprod()*(inv_real[4]),
                               'IBM': (retornos['IBM'] + 1).cumprod()*(inv_real[5]),
                               'AAL': (retornos['AAL'] + 1).cumprod()*(inv_real[6]),
                               'NVDA': (retornos['NVDA'] + 1).cumprod()*(inv_real[7]),
                               'PFE': (retornos['PFE'] + 1).cumprod()*(inv_real[8]),
                               'MSFT': (retornos['MSFT'] + 1).cumprod()*(inv_real[9]),
                               'AAPL': (retornos['AAPL'] + 1).cumprod()*(inv_real[10]),
                               'VWAGY': (retornos['VWAGY'] + 1).cumprod()*(inv_real[11]),
                               'NFLX': (retornos['NFLX'] + 1).cumprod()*(inv_real[12]),
                               'MA': (retornos['MA'] + 1).cumprod()*(inv_real[13]),
                               'KO': (retornos['KO'] + 1).cumprod()*(inv_real[14])})

    
    fig = px.line(cumrets, x = "Date", y = cumrets.columns,
              hover_data = {"Date": "|%B %d, %Y"},
              title = 'Evolución del Portafolio')
    fig.update_xaxes(
        dtick = "M1",
        tickformat="%b\n%Y")
    fig.show()
    
    # Valor Final de Inversión 
    vfinal_activos = np.sum(cumrets.tail(1).values[0][1:])
    vfinal_rf = asignacion_rf*(1+rf)   
    vfinal_pfolio = vfinal_activos + vfinal_rf

    # RETORNO DE ACCIONES %
    rnd_activos = pct_chng(vfinal_activos, np.sum(inv_real))
    rnd_rf = pct_chng(vfinal_rf, asignacion_rf)
    rnd_pfolio = pct_chng(vfinal_pfolio, inversion)
    
    Resultados = pd.DataFrame(columns=['Tipo', 'Inversión Inicial', 'Valor Final', 'Rendimiento %'])
    
    Resultados['Tipo'] = ['Activos','Tasa', 'Portafolio']
    Resultados['Inversión Inicial'] = [np.sum(inv_real), asignacion_rf, inversion]
    Resultados['Valor Final'] = [vfinal_activos, vfinal_rf, vfinal_pfolio]
    Resultados['Rendimiento %'] = [rnd_activos, rnd_rf, rnd_pfolio]
    
    return np.round(Resultados, 4)

    inv_real, asignacion_rf, vfinal_activos, vfinal_rf, vfinal_pfolio, rnd_activos, rnd_rf, rnd_pfolio, cumrets = None

In [4]:
# Cambiar estilo de gráficos a seaborn
matplotlib.style.use('seaborn')

# Cambiar motor gráfico a PLOTLY
pd.options.plotting.backend = "plotly"

In [5]:
# Función para descargar precios de cierre ajustados:
def get_adj_closes(tickers, start_date = None, end_date = None, freq = None):
    closes = web.YahooDailyReader(symbols = tickers,
                                  start = start_date,
                                  end = end_date,
                                  interval = freq).read()['Adj Close']
    return closes

In [6]:
def get_prices(tickers, start_date = None, end_date = None, period = '1d'):
    prices = pdr.get_data_yahoo(tickers, 
                                start_date, 
                                end_date, 
                                period)['Adj Close']

    return prices

In [7]:
# Función objetivo
def varianza(w,Sigma):
    return w.T.dot(Sigma).dot(w) 

In [8]:
# Función objetivo
def menos_RS(w,Eind,Sigma,rf):
    Ep = Eind.T.dot(w)
    sp = (w.T.dot(Sigma).dot(w))**.5
    RS = (Ep-rf)/sp
    return -RS

In [9]:
# Función Cambio Porcentual
def pct_chng(VA: 'Valor Actual',
             VI: 'Valor Inicial'):  
    ret = np.round(((VA-VI)/VI), 2)
    return ret

# Portafolio

In [10]:
rf = .0544/91

In [11]:
II = 100_000

In [12]:
watchlist = ['VNQ','VBK','VTI','VGT','IBM','AAL','NVDA','PFE','MSFT',
             'AAPL','VWAGY','NFLX','MA','KO','RCL','CDEV','DIS','INO',
             'ZM','VOO','UTF','TSM','PLTR','MRNA','GTHX','FDX','CCL',
             'ABNB','AMD','VUG','VGLT','VHT','FSLR','VTIP','WNS','NIO']

In [13]:
# Cantidad de acciones compradas
Q = [30, 60, 5, 10, 20, 6, 50, 30, 10, 80, 3, 20, 200, 2, 10, 20, 17]

In [14]:
# Precio promedio de compra inicial de cada acción
pprice = [120, 56.32, 345.19, 332.1, 45.31, 424.18, 52.05, 90.13, 281.36, 41.52, 262.14, 169.81, 28.55, 22.83, 125.89, 315.27, 236.73]

In [15]:
# Activos en el portafolio
pfolio_assets = 'FSLR KO MRNA MSFT PFE VOO VTIP WNS ZM NIO NVDA DIS UTF CCL AMD VUG FDX'
pfolio_assets = pfolio_assets.split(' ')

### Simulación Portafolio

In [40]:
start = "2021-10-20"
end = "2021-11-04"
freq = "d"

In [41]:
prices = get_adj_closes(pfolio_assets, start, end, freq)

In [42]:
prices

Symbols,FSLR,KO,MRNA,MSFT,PFE,VOO,VTIP,WNS,ZM,NIO,NVDA,DIS,UTF,CCL,AMD,VUG,FDX
Date,,,,,,,,,,,,,,,,,
2021-10-20,103.230003,54.630001,333.010010,307.410004,42.799999,415.989990,51.980000,86.230003,275.709991,39.779999,221.029999,170.550003,28.129999,22.440001,116.389999,306.109985,231.300003
2021-10-21,106.290001,54.349998,339.570007,310.760010,42.860001,417.010010,52.040001,84.290001,274.779999,39.970001,226.919998,171.339996,27.980000,22.330000,119.330002,308.510010,232.839996
2021-10-22,106.220001,54.450001,326.540009,309.160004,43.160000,416.589996,52.060001,84.580002,277.579987,38.880001,227.259995,169.419998,28.200001,22.290001,119.820000,306.440002,232.979996
2021-10-25,109.040001,54.230000,349.559998,308.130005,43.150002,418.750000,52.169998,84.730003,279.790009,41.270000,231.660004,172.009995,28.129999,22.049999,122.360001,308.920013,235.399994
2021-10-26,108.940002,54.470001,345.739990,310.109985,43.560001,419.149994,52.189999,85.029999,278.700012,40.470001,247.169998,172.039993,28.160000,21.820000,122.930000,309.390015,237.830002
2021-10-27,110.230003,55.520000,340.880005,323.170013,42.970001,417.329987,52.209999,84.510002,272.809998,39.310001,244.509995,169.550003,28.059999,21.639999,122.279999,309.609985,235.289993
2021-10-28,117.879997,56.040001,347.920013,324.350006,43.180000,421.410004,52.060001,89.190002,276.579987,40.790001,249.410004,169.679993,28.370001,21.969999,121.160004,312.739990,235.960007
2021-10-29,119.589996,56.369999,345.209991,331.619995,43.740002,422.160004,52.070000,88.809998,274.649994,39.410000,255.669998,169.070007,28.420000,22.160000,120.230003,314.149994,235.529999
2021-11-01,121.139999,56.169998,337.170013,329.369995,43.639999,423.049988,52.000000,89.709999,278.940002,40.840000,258.269989,170.190002,28.520000,23.070000,125.230003,314.299988,235.639999


In [19]:
# prices.to_csv('D:/Downloads/prices.csv', index = False)

In [43]:
ret = prices.pct_change().dropna()

In [44]:
ret

Symbols,FSLR,KO,MRNA,MSFT,PFE,VOO,VTIP,WNS,ZM,NIO,NVDA,DIS,UTF,CCL,AMD,VUG,FDX
Date,,,,,,,,,,,,,,,,,
2021-10-21,0.029643,-0.005125,0.019699,0.010898,0.001402,0.002452,0.001154,-0.022498,-0.003373,0.004776,0.026648,0.004632,-0.005332,-0.004902,0.025260,0.007840,0.006658
2021-10-22,-0.000659,0.001840,-0.038372,-0.005149,0.007000,-0.001007,0.000384,0.003441,0.010190,-0.027270,0.001498,-0.011206,0.007863,-0.001791,0.004106,-0.006710,0.000601
2021-10-25,0.026549,-0.004040,0.070497,-0.003332,-0.000232,0.005185,0.002113,0.001773,0.007962,0.061471,0.019361,0.015287,-0.002482,-0.010767,0.021198,0.008093,0.010387
2021-10-26,-0.000917,0.004426,-0.010928,0.006426,0.009502,0.000955,0.000383,0.003541,-0.003896,-0.019385,0.066952,0.000174,0.001067,-0.010431,0.004658,0.001521,0.010323
2021-10-27,0.011841,0.019277,-0.014057,0.042114,-0.013545,-0.004342,0.000383,-0.006115,-0.021134,-0.028663,-0.010762,-0.014473,-0.003551,-0.008249,-0.005288,0.000711,-0.010680
2021-10-28,0.069400,0.009366,0.020652,0.003651,0.004887,0.009776,-0.002873,0.055378,0.013819,0.037649,0.020040,0.000767,0.011048,0.015250,-0.009159,0.010110,0.002848
2021-10-29,0.014506,0.005889,-0.007789,0.022414,0.012969,0.001780,0.000192,-0.004261,-0.006978,-0.033832,0.025099,-0.003595,0.001762,0.008648,-0.007676,0.004509,-0.001822
2021-11-01,0.012961,-0.003548,-0.023290,-0.006785,-0.002286,0.002108,-0.001344,0.010134,0.015620,0.036285,0.010169,0.006624,0.003519,0.041065,0.041587,0.000477,0.000467
2021-11-02,-0.020555,-0.001246,0.034018,0.011416,0.041476,0.003877,0.000962,0.005908,0.004840,0.011263,0.022225,-0.002115,-0.004558,-0.006935,0.019165,0.002959,0.000552


In [52]:
a, b = simulacion_pfolios(ret, rf, pfolio_assets)

In [53]:
display(a, b)

,Media,Vol,w
Symbols,,,
FSLR,0.010655,0.025364,0.0369
KO,0.003079,0.007030,0.1000
MRNA,-0.012612,0.063668,0.0000
MSFT,0.008082,0.014047,0.1000
PFE,0.001755,0.017845,0.0890
VOO,0.002763,0.003702,0.1000
VTIP,0.000333,0.001434,0.1000
WNS,0.003177,0.020130,0.0000
ZM,0.002938,0.011978,0.0261


,E_EMV,Volatilidad,Sharpe Ratio
0,0.005074,0.001928,2.32189


In [50]:
corr

Symbols,FSLR,KO,MRNA,MSFT,PFE,VOO,VTIP,WNS,ZM,NIO,NVDA,DIS,UTF,CCL,AMD,VUG,FDX
Symbols,,,,,,,,,,,,,,,,,
FSLR,1.000000,0.128331,0.377510,-0.013490,-0.044158,0.380935,-0.558677,0.576053,0.072502,0.302423,-0.223200,0.290939,0.471778,0.268220,-0.390690,0.365066,-0.191667
KO,0.128331,1.000000,-0.189409,0.725564,-0.272796,-0.327434,-0.295461,0.222327,-0.525712,-0.465004,-0.140155,-0.711643,0.152119,-0.135226,-0.660065,-0.089671,-0.493863
MRNA,0.377510,-0.189409,1.000000,0.024778,0.591739,0.195449,-0.206953,0.019311,0.202854,0.048663,-0.676742,0.443604,0.257305,0.118658,-0.367119,-0.023716,-0.076169
MSFT,-0.013490,0.725564,0.024778,1.000000,-0.021560,-0.518847,0.083332,-0.247869,-0.842048,-0.566610,-0.180825,-0.574063,-0.347431,-0.314677,-0.506600,-0.012482,-0.608548
PFE,-0.044158,-0.272796,0.591739,-0.021560,1.000000,0.079792,-0.135780,0.159240,0.113470,-0.192378,-0.272340,0.061513,0.186539,0.003094,-0.312331,-0.287450,-0.247368
VOO,0.380935,-0.327434,0.195449,-0.518847,0.079792,1.000000,-0.303852,0.500539,0.670691,0.684657,0.186557,0.606985,0.257846,0.246564,0.151328,0.714369,0.580156
VTIP,-0.558677,-0.295461,-0.206953,0.083332,-0.135780,-0.303852,1.000000,-0.666323,-0.381069,0.010626,0.383869,0.062691,-0.814267,-0.746438,0.388650,0.077634,0.266719
WNS,0.576053,0.222327,0.019311,-0.247869,0.159240,0.500539,-0.666323,1.000000,0.309570,0.389736,0.079784,0.033241,0.596727,0.282972,-0.292783,0.164054,-0.175741
ZM,0.072502,-0.525712,0.202854,-0.842048,0.113470,0.670691,-0.381069,0.309570,1.000000,0.504488,-0.186635,0.501233,0.537400,0.564426,0.318419,0.053889,0.517554


### Portafolio Real

In [24]:
n_assets = len(pfolio_assets)
n_assets

17

In [25]:
actual_prices = get_adj_closes(pfolio_assets, '2021-11-03', '2021-11-03', 'd')

In [26]:
actual_price = np.round(actual_prices.head(1).values[0], 2)

In [27]:
pct_change = np.round([actual_price[i]/pprice[i]-1 for i in range(n_assets)], 4)
pct_change

array([-0.0264, -0.0005,  0.0021,  0.0057, -0.0108,  0.0072,  0.0002,
       -0.0171,  0.0153, -0.0005,  0.0146,  0.0016, -0.0042,  0.0149,
        0.0369,  0.0066,  0.0139])

In [28]:
initial_pvalue = np.sum([pprice[i]*Q[i] for i in range(n_assets)])

In [29]:
initial_pvalue

48446.020000000004

In [30]:
# Cantidad en USD disponible
USD = II-initial_pvalue
USD

51553.979999999996

In [31]:
# Balance Inicial
IB = USD+initial_pvalue
IB

100000.0

In [32]:
actual_pvalue = np.sum([(pprice[i]*Q[i])*(1+pct_change[i]) for i in range(n_assets)])
actual_pvalue

48513.67563600001

In [33]:
Actual_Balance = actual_pvalue+USD
Actual_Balance

100067.65563600001

In [34]:
w = np.round([(Q[i]*actual_price[i])/Actual_Balance for i in range(n_assets)], 4)
w

array([0.035 , 0.0338, 0.0173, 0.0334, 0.009 , 0.0256, 0.026 , 0.0266,
       0.0285, 0.0332, 0.008 , 0.034 , 0.0568, 0.0005, 0.013 , 0.0634,
       0.0408])

In [35]:
pfolio = pd.DataFrame(data = {'Ticker': pfolio_assets,
                              'Q': Q,
                              'Purchase price': pprice,
                              'Actual Price': actual_price,
                              '% change': pct_change*100,
                              'w': w*100})
pfolio

,Ticker,Q,Purchase price,Actual Price,% change,w
0,FSLR,30,120.00,116.83,-2.64,3.50
1,KO,60,56.32,56.29,-0.05,3.38
2,MRNA,5,345.19,345.92,0.21,1.73
3,MSFT,10,332.10,334.00,0.57,3.34
4,PFE,20,45.31,44.82,-1.08,0.90
5,VOO,6,424.18,427.25,0.72,2.56
6,VTIP,50,52.05,52.06,0.02,2.60
7,WNS,30,90.13,88.59,-1.71,2.66
8,ZM,10,281.36,285.66,1.53,2.85
9,NIO,80,41.52,41.50,-0.05,3.32


In [36]:
np.round((((actual_pvalue+USD)/(initial_pvalue+USD))-1), 6)

0.000677

In [37]:
pfolio_R = np.mean(pct_change)
pfolio_R

0.0034999999999999996

In [38]:
pfolio_std = np.std(pct_change)
pfolio_std

0.013879481258317977

In [39]:
pfolio_RS = (pfolio_R-rf)/pfolio_std
pfolio_RS

0.20909987543363795